# Detectron 학습
mAP50: 0.83    
mAP50-95: 0.475

In [1]:
cd ..

c:\Users\USER\Desktop\train


In [2]:
import detectron2
from detectron2.engine import DefaultTrainer, HookBase
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.config import get_cfg
from detectron2 import model_zoo

## 데이터 설정

In [3]:
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import load_coco_json

In [4]:
# 학습 데이터 경로
train_json_file = r"C:\Users\USER\Desktop\train\dataset\annotations\train_annotations.coco.json"
# 검증 데이터 경로
val_json_file = r'C:\Users\USER\Desktop\train\dataset\annotations\val_annotations.coco.json'
# 이미지 폴더 경로
image_root = r"C:\Users\USER\Desktop\train\dataset\images"

In [5]:
# 데이터셋 등록 함수
def register_coco_instances(name, json_file, image_root):
    DatasetCatalog.register(name, lambda: load_coco_json(json_file, image_root, name))
    MetadataCatalog.get(name).set(json_file=json_file, image_root=image_root, evaluator_type="coco")

In [6]:
# COCO 데이터셋 등록
register_coco_instances("my_coco_train", train_json_file, image_root)
register_coco_instances("my_coco_val", val_json_file, image_root)

In [7]:
# 등록된 학습 데이터셋 확인
train_dataset_dicts = DatasetCatalog.get("my_coco_train")
print(f"Training dataset samples: {len(train_dataset_dicts)}")
print(train_dataset_dicts[:2])  # 첫 두 개의 샘플 확인

# 등록된 검증 데이터셋 확인
val_dataset_dicts = DatasetCatalog.get("my_coco_val")
print(f"Validation dataset samples: {len(val_dataset_dicts)}")
print(val_dataset_dicts[:2])  # 첫 두 개의 샘플 확인

Training dataset samples: 777
[{'file_name': 'C:\\Users\\USER\\Desktop\\train\\dataset\\images\\3-single_fire_136_jpg.rf.0008541daf4e4ed061960779b68dfd0a.jpg', 'height': 640, 'width': 640, 'image_id': 0, 'annotations': [{'iscrowd': 0, 'bbox': [431, 467, 33.5, 60], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [460, 340, 117.5, 158], 'category_id': 2, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}]}, {'file_name': 'C:\\Users\\USER\\Desktop\\train\\dataset\\images\\2_mountain_lake_land_sunset_0410_jpg.rf.005db329dae369f0f6fdd0438e0319cb.jpg', 'height': 640, 'width': 640, 'image_id': 1, 'annotations': [{'iscrowd': 0, 'bbox': [469, 174, 73.5, 81], 'category_id': 1, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}, {'iscrowd': 0, 'bbox': [465, 66, 80.5, 112.5], 'category_id': 2, 'bbox_mode': <BoxMode.XYWH_ABS: 1>}]}]
Validation dataset samples: 381
[{'file_name': 'C:\\Users\\USER\\Desktop\\train\\dataset\\images\\2_mountain_lake_land_night_0450_jpg.rf.0216d77782a1705af06777cf2ceee19

## 학습 설정

In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("my_coco_train",)  # 등록한 학습 데이터셋
cfg.DATASETS.TEST = ("my_coco_val",)     # 등록한 검증 데이터셋

cfg.SOLVER.WARMUP_ITERS = 100
# 1 epoch = 388.5 iter(학습 이미지 수 / 배치 크기)
# 50 epoch = 약 19600 iter
cfg.SOLVER.IMS_PER_BATCH = 2 # 이미지 배치 수
cfg.SOLVER.BASE_LR = 0.001  # 학습률
cfg.SOLVER.MAX_ITER = 19600  # 최대 학습 스텝
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16  # RoI 예측에 사용될 배치 사이즈
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 클래스 개수 (클래스가 2개인 경우에도 배경 포함 3으로 설정)

In [9]:
# 학습 실행
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[10/19 17:28:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[10/19 17:28:03 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\USER\anaconda3\envs\detectron\lib\site-packages\detectron2\structures\image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
c:\Users\USER\anaconda3\envs\detectron\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/19 17:28:15 d2.utils.events]:  eta: 1:31:15  iter: 19  total_loss: 2.472  loss_cls: 1.211  loss_box_reg: 0.8458  loss_rpn_cls: 0.2828  loss_rpn_loc: 0.04982  time: 0.2801  data_time: 0.1365  lr: 0.00019081  max_mem: 3149M
[10/19 17:28:21 d2.utils.events]:  eta: 1:31:58  iter: 39  total_loss: 1.78  loss_cls: 0.7039  loss_box_reg: 0.9463  loss_rpn_cls: 0.0654  loss_rpn_loc: 0.05869  time: 0.2865  data_time: 0.0022  lr: 0.00039061  max_mem: 3149M
[10/19 17:28:26 d2.utils.events]:  eta: 1:31:00  iter: 59  total_loss: 1.543  loss_cls: 0.5424  loss_box_reg: 0.8975  loss_rpn_cls: 0.05115  loss_rpn_loc: 0.04316  time: 0.2802  data_time: 0.0017  lr: 0.00059041  max_mem: 3149M
[10/19 17:28:31 d2.utils.events]:  eta: 1:28:49  iter: 79  total_loss: 1.35  loss_cls: 0.4071  loss_box_reg: 0.8859  loss_rpn_cls: 0.0368  loss_rpn_loc: 0.05078  time: 0.2743  data_time: 0.0017  lr: 0.00079021  max_mem: 3149M
[10/19 17:28:36 d2.utils.events]:  eta: 1:28:00  iter: 99  total_loss: 1.211  loss_cls: 0.3395

## mAP 확인

In [10]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, DatasetEvaluators
from detectron2.data import build_detection_test_loader

# 학습이 완료된 후 평가를 수행하기 위한 코드
cfg.MODEL.EVALUATOR_TYPE = "coco"
evaluator = COCOEvaluator("my_coco_val", cfg, False, output_dir="./output/")

# 검증 데이터 로더 설정
val_data_loader = build_detection_test_loader(cfg, "my_coco_val")

# 평가 수행
metrics = inference_on_dataset(trainer.model, val_data_loader, evaluator)

# 평가 결과 출력
print(metrics)

WARNING [10/19 18:55:39 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[10/19 18:55:39 d2.data.datasets.coco]: Loaded 381 images in COCO format from C:\Users\USER\Desktop\train\dataset\annotations\val_annotations.coco.json
[10/19 18:55:39 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[10/19 18:55:39 d2.data.common]: Serializing 381 elements to byte tensors and concatenating them all ...
[10/19 18:55:39 d2.data.common]: Serialized dataset takes 0.15 MiB
[10/19 18:55:39 d2.evaluation.evaluator]: Start inference on 381 batches
[10/19 18:55:43 d2.evaluation.evaluator]: Inference done 11/381. Dataloading: 0.0003 s/iter. Inference: 0.0405 s/iter. Eval: 0.0001 s/iter. Total: 0.0410 s/iter. ETA=0:00:15
[10/19 18:55:48 d2.evaluation.evaluator]: Inference done 133/381. Dataloading: 0

# YOLO 버전별 비교

## YOLOv5n
mAP50: 0.821   
mAP50-95: 0.451

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov5n.pt')

In [ ]:
import os
os.environ['WANDB_MODE'] = 'dryrun'

# 모델 훈련
model.train(data='/content/drive/MyDrive/data.yaml', epochs=100, batch=16)  # 'dataset.yaml' 파일에 데이터셋 정보가 있어야 합니다.

Ultralytics 8.3.18 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov5n.pt, data=/content/drive/MyDrive/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sh

100%|██████████| 755k/755k [00:00<00:00, 19.1MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1760  ultralytics.nn.modules.conv.Conv             [3, 16, 6, 2, 2]              
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      4800  ultralytics.nn.modules.block.C3              [32, 32, 1]                   
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     29184  ultralytics.nn.modules.block.C3              [64, 64, 2]                   
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  3    156928  ultralytics.nn.modules.block.C3              [128, 128, 3]                 
  7                  -1  1    295424  ultralytics

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Freezing layer 'model.24.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...


...


100 epochs completed in 0.719 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 5.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 5.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.18 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv5n summary (fused): 193 layers, 2,503,334 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.74it/s]


                   all        381       1597      0.778      0.774      0.821      0.451
                  fire        311        828      0.806      0.815      0.855      0.443
                 smoke        314        769       0.75      0.732      0.788      0.459
Speed: 0.3ms preprocess, 2.7ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/train2


lr/pg0,▃███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
lr/pg1,▃████▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▁▁
lr/pg2,▃██▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▂▁▃▄▄▃▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▇▇▇█▇▇▇█▇██████████
metrics/mAP50-95(B),▁▁▂▄▄▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇██▇█████████████
metrics/precision(B),▁▃▄▂▅▄▅▅▆▆▆▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇██████████
metrics/recall(B),▁▅▅▆▇▆▆▇▇▇▇▇▇▇▇█▇█▇█████████████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ce978562770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

## YOLOv6n
mAP50: 0.642      
mAP50-95: 0.316

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov6n.yaml')  # 사전 훈련된 모델 사용

In [ ]:
model.train(data='/content/drive/MyDrive/data.yaml', epochs=100, batch=2)

## YOLOv8n
mAP50: 0.824   
mAP50-95: 0.468

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')  # 사전 훈련된 모델 사용

In [ ]:
import os
os.environ['WANDB_MODE'] = 'dryrun'

# 모델 훈련
model.train(data='/content/drive/MyDrive/data.yaml', epochs=100, batch=8)  # 'dataset.yaml' 파일에 데이터셋 정보가 있어야 합니다.

Ultralytics 8.3.17 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 755k/755k [00:00<00:00, 32.8MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...


...

                   all        381       1597      0.792      0.762      0.829      0.464

100 epochs completed in 0.826 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.3MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.17 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:06<00:00,  3.95it/s]


                   all        381       1597      0.805      0.752      0.824      0.468
                  fire        311        828      0.839      0.801      0.864      0.458
                 smoke        314        769       0.77      0.704      0.783      0.478
Speed: 0.4ms preprocess, 3.3ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/train2


lr/pg0,█▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁
lr/pg1,▃████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▆█████▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▁▁▁
metrics/mAP50(B),▁▂▂▄▄▅▅▆▆▆▇▇▆▆▇▆▇▇▇▇▇▇▇█▇███████████████
metrics/mAP50-95(B),▁▂▄▄▅▅▅▅▆▆▆▆▆▆▇▇▆▇▇▇▇▇▆█▇▇█▇▇▇██████████
metrics/precision(B),▂▁▄▅▆▆▆▆▇▇▇▆▇▆▇▇▇▇▇█▇▇▇█▇▇▇█████████████
metrics/recall(B),▁▄▄▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█▇██▇██████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a38f5757160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804